### QA notebook - DP0 object selection
#### Andressa Wille
#### 19/07/22

Plots: Plot RA x Dec; Distribuições de magnitudes; Distribuições de cores; Distribuições de photo-z; Distribuições de photo-z error; Number counts normalizados com referências da literatura (vide product log); Plot magnitude X redshift; Color-color diagrams.


To do:
Plot proj distribution stars, galaxies, highlight bright objects → plot ra x dec (<12, <13, <14..) stars and galaxies separately
Number counts without cuts (hist normalized by area, log scale)
Create a list of very bright objects to explore on RSP
Explore DP0 images - overlay bright objects on images (in prep. for a future foreground objects mask)
Color distributions - objects around bright objects might have funny color


Dados: DP0


### Imports

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
import pylab as pl
from astropy.table import Table
from astropy import units as u
from astropy.coordinates import SkyCoord
import sys
from obj_sel_funcs import *
from gavodb import DBManager

%load_ext autoreload
%autoreload 2

In [ ]:
path = '/archive/user/aguena/cats/dp0/wazp/7038/'

In [ ]:
wazp_membership = Table.read(path + 'wazp_membership.fits', format='fits')
print(wazp_membership.colnames)

In [ ]:
bands = ['g', 'r', 'i', 'z', 'y']
colors = ['g-r', 'r-i', 'i-z', 'z-y']

### Plot RA x Dec

In [ ]:
mag_min = 14
mag_min_mask =  wazp_membership[f'mag_g'] > mag_min
#qual banda?
#como separar estrelas e galáxias?
ra =  wazp_membership['RA'][mag_min_mask]
ra[ra >180.0] -= 360
dec =  wazp_membership['DEC'][mag_min_mask]
plt.hist2d(ra, dec, bins=50, weights=None, cmin=0.5, cmax=None,cmap='viridis')
plt.xlabel("RA (deg)", fontsize=14)
plt.ylabel("Dec (deg)", fontsize=14)
plt.colorbar(label="density of points")
ylims=[-60, -10]
plt.ylim(ylims)
xlims=[0, 100]
plt.xlim(xlims)
plt.grid()
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()

In [ ]:
foot_ra, foot_dec = np.loadtxt('des-round17-poly.txt', unpack=True)

ra=-np.array(wazp_membership['RA'])*u.degree
dec=np.array(wazp_membership['DEC'])*u.degree

c = SkyCoord(ra, dec, frame='icrs')
ra_rad = c.ra.wrap_at(180 * u.deg).radian
dec_rad = c.dec.radian
plot_specz_spatial_dist(ra_rad, dec_rad, foot_ra, foot_dec, projection='mollweide', 
                        cmap='viridis', cbar_label='log(#spec-$z$/deg$^{2}$)')

### Distribuição de magnitudes

In [ ]:
for i, band in enumerate(bands):
    plt.figure()
    plt.hist(wazp_membership[f'mag_{band}'], histtype='bar', bins='auto', ec="cornflowerblue")
    plt.xlabel("mag "+band)
    plt.ylabel("Counts")
    plt.xlim(10, 40)
    plt.yscale('log')
    plt.grid(True)
    plt.tight_layout()

### Distribuição de cores

In [ ]:
for i, band in enumerate(colors):
    plt.figure() 
    wazp_membership[colors[i]] = wazp_membership[f'mag_{bands[i]}'] - wazp_membership[f'mag_{bands[i+1]}']
    plt.hist(wazp_membership[colors[i]], bins='auto', ec="cornflowerblue", fc="cornflowerblue")
    plt.xlabel(band)
    plt.ylabel("Counts")
    plt.yscale('log')
    plt.grid(True)
    plt.tight_layout()

### Distribuição de redshift

In [ ]:
plt.hist(wazp_membership['ZP'], bins='auto', ec="cornflowerblue", fc="cornflowerblue")
plt.xlabel("z")
plt.ylabel("Counts")
plt.title("Redshift Distribution")
plt.grid(True)
plt.tight_layout()

### Distribuição de riqueza

In [ ]:
plt.hist(wazp_membership['NGALS'], bins='auto', ec="cornflowerblue", fc="cornflowerblue")
plt.xlabel("$N_{gals}$")
plt.ylabel("Counts")
plt.title("Richness Distribution")
plt.grid(True)
plt.tight_layout()

### Number counts

In [ ]:
area = 300 #??
bins = np.linspace(15,25,21)
for i, band in enumerate(bands):
    plt.figure()
    n, bins, patches = plt.hist(wazp_membership[f'mag_{band}'], histtype='step', weights=(wazp_membership[f'mag_{band}']/area),
             bins=bins, color='darkblue')
    #print(np.argmax(n))
    #print(bins)
    maximo = (bins[np.argmax(n)] + bins[np.argmax(n)+1])/2
    plt.vlines(maximo, 10e-1, 10e6, linestyles='dotted')
    plt.text(maximo-1, 10e4, maximo, fontsize=11)
    plt.xlabel("mag "+band)
    plt.ylabel("$N_{gal} deg^{-2}0.5mag^{-1}$")
    plt.xlim(15, 25)
    plt.ylim(10e-1, 10e5)
    plt.yscale('log')
    plt.grid(True)
    plt.tight_layout()

### Magnitude x redshift

In [ ]:
for i, band in enumerate(bands):
    pl.figure(figsize=[10,4])
    pl.subplot(121) 
    mag = np.array(wazp_membership[f'mag_{band}'])
    z = np.array(wazp_membership['ZP'])
    mask = (mag>14)&(mag<32)&(z<=2)
    pl.hexbin(z[mask], mag[mask], None,  mincnt=1, cmap='viridis', gridsize=[200,100], bins='log')
    cbar = pl.colorbar()
    pl.xlabel("mag "+band)
    pl.ylabel("redshift")
    pl.grid(True)
    pl.tight_layout()

### Diagrama cor-cor

In [ ]:
CCD(wazp_membership['mag_g'], wazp_membership['mag_r'], wazp_membership['mag_i'], wazp_membership['mag_z'], 'g', 'r', 'i', 'z', 24.)